# 03 - Image Classification Experiments

This notebook develops and evaluates the CNN-based image classifier for detecting rat evidence.

**Classes:**
- rat (actual rat sighting)
- droppings
- burrow
- gnaw_marks
- no_evidence

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

from src.image_classifier import (
    RatEvidenceClassifier, ImageClassifierTrainer,
    load_classifier, classify_image
)
from src import config

%matplotlib inline

## 1. Model Architecture

In [ ]:
# Create model
model = RatEvidenceClassifier(
    num_classes=5,
    architecture='resnet18',
    pretrained=True,
    dropout=0.5
)

print("Model Architecture:")
print(f"Backbone: ResNet-18 (pretrained on ImageNet)")
print(f"Classes: {config.IMAGE_CLASSES}")
print(f"\nClassifier Head:")
print(model.classifier)

In [ ]:
# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## 2. Data Augmentation Pipeline

In [ ]:
import albumentations as A

# Define augmentation pipeline
aug_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussianBlur(blur_limit=3, p=0.3),
    A.RandomResizedCrop(height=224, width=224, scale=(0.8, 1.0), p=0.5),
])

print("Augmentation Pipeline:")
for t in aug_transform.transforms:
    print(f"  - {t.__class__.__name__}")

In [ ]:
# Visualize augmentation (with synthetic image)
sample_image = np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8)

fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes[0, 0].imshow(sample_image)
axes[0, 0].set_title('Original')
axes[0, 0].axis('off')

for i, ax in enumerate(axes.flat[1:]):
    augmented = aug_transform(image=sample_image)['image']
    ax.imshow(augmented)
    ax.set_title(f'Augmented {i+1}')
    ax.axis('off')

plt.suptitle('Data Augmentation Examples')
plt.tight_layout()
plt.show()

## 3. Transfer Learning Strategy

We use a two-stage training approach:
1. **Stage 1**: Freeze backbone, train only classifier head (5 epochs)
2. **Stage 2**: Unfreeze backbone, fine-tune entire network

In [ ]:
# Demonstrate freezing
model.freeze_backbone()
trainable_frozen = sum(p.numel() for p in model.parameters() if p.requires_grad)

model.unfreeze_backbone()
trainable_unfrozen = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Trainable params (frozen backbone): {trainable_frozen:,}")
print(f"Trainable params (unfrozen): {trainable_unfrozen:,}")

## 4. Model Inference Demo

In [ ]:
# Create trainer with pretrained model
trainer = ImageClassifierTrainer(model, class_names=config.IMAGE_CLASSES)

# Test inference on random image
test_image = Image.fromarray(np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8))
pred_class, confidence, probs = trainer.predict(test_image)

print(f"Predicted class: {pred_class}")
print(f"Confidence: {confidence:.2%}")
print(f"\nAll probabilities:")
for cls, prob in sorted(probs.items(), key=lambda x: x[1], reverse=True):
    print(f"  {cls}: {prob:.2%}")

In [ ]:
# Visualize prediction
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

ax1.imshow(test_image)
ax1.set_title(f'Prediction: {pred_class}\nConfidence: {confidence:.2%}')
ax1.axis('off')

classes = list(probs.keys())
probabilities = list(probs.values())
colors = ['green' if c == pred_class else 'steelblue' for c in classes]

ax2.barh(classes, probabilities, color=colors)
ax2.set_xlabel('Probability')
ax2.set_title('Class Probabilities')
ax2.set_xlim(0, 1)

plt.tight_layout()
plt.show()

## 5. Expected Performance

Based on transfer learning from ImageNet and fine-tuning on rat evidence data,
we expect the following approximate performance:

| Class | Precision | Recall | F1-Score |
|-------|-----------|--------|----------|
| Rat | 0.89 | 0.85 | 0.87 |
| Droppings | 0.82 | 0.78 | 0.80 |
| Burrow | 0.86 | 0.83 | 0.84 |
| Gnaw Marks | 0.79 | 0.75 | 0.77 |
| No Evidence | 0.91 | 0.94 | 0.92 |

In [ ]:
# Simulated confusion matrix for visualization
# (Would be actual results with real training data)
simulated_cm = np.array([
    [85, 5, 3, 4, 3],
    [8, 78, 5, 6, 3],
    [4, 6, 83, 4, 3],
    [6, 7, 6, 75, 6],
    [2, 2, 2, 2, 92],
])

plt.figure(figsize=(8, 6))
sns.heatmap(
    simulated_cm, 
    annot=True, 
    fmt='d',
    cmap='Blues',
    xticklabels=config.IMAGE_CLASSES,
    yticklabels=config.IMAGE_CLASSES
)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Simulated)')
plt.tight_layout()
plt.show()

## 6. Model Interpretability with Grad-CAM

Grad-CAM helps visualize what the model is "looking at" when making predictions.

In [ ]:
# Grad-CAM implementation (simplified)
def get_gradcam(model, image_tensor, target_class):
    """Generate Grad-CAM heatmap."""
    model.eval()
    
    # Hook to capture activations and gradients
    activations = None
    gradients = None
    
    def forward_hook(module, input, output):
        nonlocal activations
        activations = output.detach()
    
    def backward_hook(module, grad_input, grad_output):
        nonlocal gradients
        gradients = grad_output[0].detach()
    
    # Register hooks on last conv layer
    target_layer = model.backbone.layer4[-1]
    fh = target_layer.register_forward_hook(forward_hook)
    bh = target_layer.register_full_backward_hook(backward_hook)
    
    # Forward pass
    output = model(image_tensor)
    
    # Backward pass
    model.zero_grad()
    output[0, target_class].backward()
    
    # Compute Grad-CAM
    weights = gradients.mean(dim=[2, 3], keepdim=True)
    cam = (weights * activations).sum(dim=1, keepdim=True)
    cam = torch.relu(cam)
    cam = cam - cam.min()
    cam = cam / cam.max()
    
    # Cleanup
    fh.remove()
    bh.remove()
    
    return cam.squeeze().cpu().numpy()

print("Grad-CAM implementation ready")

## 7. Save Model

In [ ]:
from pathlib import Path

save_dir = Path('../models/classifier')
save_dir.mkdir(parents=True, exist_ok=True)

trainer.save(str(save_dir / 'classifier.pt'))
print(f"Model saved to {save_dir / 'classifier.pt'}")